# Westeros Tutorial - Flexible generation

In the previous part, we showed how to introduce emissions into a stylized energy systems model, and what happens if you put a constraint on total CO2 emissions.

In this tutorial notebook, we will illustrate how to add other considerations, in particular the requirement to have sufficient flexible generation capacity to mitigate short-term demand fluctuations.

In [ ]:
import pandas as pd
import ixmp
import message_ix

from message_ix.utils import make_df

%matplotlib inline

In [ ]:
mp = ixmp.Platform(dbtype='HSQLDB')

In [ ]:
base = message_ix.Scenario(mp, model='Westeros Electrified', scenario='baseline')

In [ ]:
model = 'Westeros Electrified'
scen = base.clone(model, 'flexibile_generation',
                  'illustration of flexible-generation formulation',
                  keep_solution=False)
scen.check_out()

year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'

## Add a carbon tax

Note that the example below is not feasible with a strict bound on emissions, hence this tutorial uses a tax.

Below, we repeat the set-up of the emissions formulation from the "emissions bounds" tutorial.

In [ ]:
# first we introduce the emission specis CO2 and the emission category GHG
scen.add_set('emission', 'CO2')
scen.add_cat('emission', 'GHG', 'CO2')

# we now add CO2 emissions to the coal powerplant
base_emission_factor = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'unit': 'USD/GWa',
}

emission_factor = make_df(base_emission_factor, technology= 'coal_ppl', emission= 'CO2', value = 100)
scen.add_par('emission_factor', emission_factor)

In [ ]:
base_tax_emission = {
    'node': country,
    'type_year': [700,710,720],
    'type_tec': 'all',
    'unit': 'tCO2',
    'type_emission': 'GHG',
    'value': [1., 2., 3.]
}

tax_emission = make_df(base_tax_emission)
scen.add_par('tax_emission', tax_emission)

## Describing the Renewable Technologies  Flexibility 

### Flexibility of demand and supply

The wind power plant has a flexibility demand of 5% of its activity. The coal powerplant can provide 20% of its activity as flexibility.

In [ ]:
base_flexibility_factor = pd.DataFrame({
        'node_loc': country,
        'commodity': 'electricity',
        'level' : 'secondary',
        'mode': 'standard',
        'unit': '???',
        'time': 'year',
        'year_vtg': vintage_years,
        'year_act': act_years,
})

base_rating = pd.DataFrame({
        'node': country,
        'commodity': 'electricity',
        'level' : 'secondary', 
        'unit': '???',
        'time': 'year',
        'year_act': model_horizon})

In [ ]:
# add the ratings as a set 
scen.add_set('rating', ['r1', 'r2'])

# For the Load 
flexibility_factor = make_df(base_flexibility_factor, technology= 'grid', rating= 'unrated', value = -0.1)
scen.add_par('flexibility_factor',flexibility_factor)

# For the Wind PPL
rating_bin = make_df(base_rating, technology= 'wind_ppl', value = 0.2, rating= 'r1')
scen.add_par('rating_bin', rating_bin)

flexibility_factor = make_df(base_flexibility_factor, technology= 'wind_ppl', rating= 'r1', value = -0.2)
scen.add_par('flexibility_factor',flexibility_factor)

rating_bin = make_df(base_rating, technology= 'wind_ppl', value = 0.8, rating= 'r2')
scen.add_par('rating_bin', rating_bin)

flexibility_factor = make_df(base_flexibility_factor, technology= 'wind_ppl', rating= 'r2', value = -0.7)
scen.add_par('flexibility_factor',flexibility_factor)

# For the Coal PPL
flexibility_factor = make_df(base_flexibility_factor, technology= 'coal_ppl', rating= 'unrated', value = 1)
scen.add_par('flexibility_factor',flexibility_factor)

## Commit and Solve

In [ ]:
scen.commit(comment='define parameters for flexibile-generation implementation')
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var('OBJ')['lvl']

## Plotting

In [ ]:
from tools import Plots
p = Plots(scen, country, firstyear=700)

In [ ]:
p.plot_activity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

In [ ]:
p.plot_capacity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

In [ ]:
p.plot_prices(subset=['light'], baseyear=True)

## Close the connection to the database

In [ ]:
mp.close_db()